In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import gensim
import re

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [87]:
df = pd.read_csv(r'Data/dataset.csv',names=['Author','Text'])

In [88]:
df['Text'] = df['Text'].map(lambda x: re.sub('\r|\n|\'','',x))
df['Text'] = df['Text'].map(lambda x: re.sub(r'--\d\d\d-\d\d\d-\d\d\d\d','',x))

In [89]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words_subset = set([x for x in stop_words if 3 <= len(x) <= 5])

In [90]:
#text_copy = df['Text'].copy()

In [91]:
def remove_stopwords(x): #x is the string input
    word_set = x.split(' ')
    for word in word_set:
        if word in stop_words:
            word_set.remove(word)
    return ' '.join(word_set)
            


In [92]:
# Removing stopwords
df['Text'] = df['Text'].map(lambda x: remove_stopwords(x))

In [93]:
#Stratified train-test split 

X = df['Text']
y = df['Author']

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25,
                                                   random_state= 42)

In [10]:
#1. Using sklearn's CountVectorizer(i.e. A Bag of words model) to get a feature vector for our article text

In [54]:
#Building a simple bag-of-words model

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.1, lowercase=False) #Only consider words with a minimum frequency of 0.1, dont convert words to lowercase

In [55]:
vectorizer.fit(df['Text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=0.1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [56]:
len(vectorizer.vocabulary_) #The vector has length of 444 i.e. the top 444 words in the corpus have been taken to make feature vectors out of

444

In [57]:
X_train = vectorizer.transform(X_train.values)

In [58]:
X_test = vectorizer.transform(X_test.values)

In [59]:
(X_test).toarray().shape

(1250, 444)

In [60]:
#Building the neural network model. For starters, we try a simple shallow NN and observe the outcome

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [61]:
input_dimension = X_train.shape[1] #This specifies the number of neurons in the input layer, in this case 444

#Using Keras' sequential API
model = Sequential()
model.add(layers.Dense(100, input_dim=input_dimension, activation='relu'))
model.add(layers.Dense(50, activation='softmax'))

In [62]:
model.compile(loss='categorical_crossentropy', 
               optimizer= 'adam', 
            metrics=['accuracy'])

#Using Adam optimizer and the categorical cross-entropy loss function

In [63]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               44500     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
Total params: 49,550
Trainable params: 49,550
Non-trainable params: 0
_________________________________________________________________


In [64]:
#The target column has string categorical values. Convert this to discrete numeric values using Keras' LabelEncoder
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(df['Author'].unique())

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [65]:
#encoder.transform(df['Author'])
#author_copy = df['Author'].copy()

In [66]:
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [67]:
history = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=True,
                     validation_data=(X_test, y_test),
                    batch_size=10)

Train on 3750 samples, validate on 1250 samples
Epoch 1/100
3750/3750 [==============================] - 4s 980us/step - loss: 2.8268 - acc: 0.2933 - val_loss: 1.7528 - val_acc: 0.5088
Epoch 2/100
3750/3750 [==============================] - 3s 748us/step - loss: 1.1412 - acc: 0.6984 - val_loss: 1.2099 - val_acc: 0.6568
Epoch 3/100
3750/3750 [==============================] - 3s 749us/step - loss: 0.6108 - acc: 0.8475 - val_loss: 1.0404 - val_acc: 0.7000
Epoch 4/100
3750/3750 [==============================] - 3s 898us/step - loss: 0.3536 - acc: 0.9296 - val_loss: 0.9367 - val_acc: 0.7376
Epoch 5/100
3750/3750 [==============================] - 3s 781us/step - loss: 0.2092 - acc: 0.9685 - val_loss: 0.9070 - val_acc: 0.7456
Epoch 6/100
3750/3750 [==============================] - 3s 848us/step - loss: 0.1256 - acc: 0.9861 - val_loss: 0.9066 - val_acc: 0.7512
Epoch 7/100
3750/3750 [==============================] - 3s 753us/step - loss: 0.0784 - acc: 0.9939 - val_loss: 0.8891 - val_acc: 

3750/3750 [==============================] - 3s 748us/step - loss: 5.4626e-05 - acc: 1.0000 - val_loss: 1.5527 - val_acc: 0.7448
Epoch 60/100
3750/3750 [==============================] - 3s 721us/step - loss: 4.7058e-05 - acc: 1.0000 - val_loss: 1.5673 - val_acc: 0.7456
Epoch 61/100
3750/3750 [==============================] - 3s 728us/step - loss: 4.1081e-05 - acc: 1.0000 - val_loss: 1.5782 - val_acc: 0.7464
Epoch 62/100
3750/3750 [==============================] - 3s 739us/step - loss: 3.4763e-05 - acc: 1.0000 - val_loss: 1.5841 - val_acc: 0.7464
Epoch 63/100
3750/3750 [==============================] - 3s 727us/step - loss: 3.0045e-05 - acc: 1.0000 - val_loss: 1.5960 - val_acc: 0.7456
Epoch 64/100
3750/3750 [==============================] - 3s 740us/step - loss: 2.5529e-05 - acc: 1.0000 - val_loss: 1.6040 - val_acc: 0.7448
Epoch 65/100
3750/3750 [==============================] - 3s 742us/step - loss: 2.1714e-05 - acc: 1.0000 - val_loss: 1.6129 - val_acc: 0.7448
Epoch 66/100
3750/3

In [68]:
#dir(history)
#history.model
history.params

{'batch_size': 10,
 'epochs': 100,
 'steps': None,
 'samples': 3750,
 'verbose': True,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc'],
 'validation_steps': None}

In [69]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=True)

3750/3750 [==============================] - 1s 179us/step


In [70]:
print(f'On the training set: loss of {loss} and accuracy achieved: {accuracy}')

On the training set: loss of 0.00430798373334037 and accuracy achieved: 0.9997333333333334


In [71]:
#A validation set accuracy of 100% suggests that our model has overfitted. 
#The accuracy on the testing set comes out to be:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7384


NEURAL NETWORK WITH A SINGLE HIDDEN LAYER AND SOFTMAX AT THE OUTPUT LAYER WITH THE COUNT VECTORIZER ACHIEVES 73.84% ACCURACY

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,2),max_features = 1000)
vectorizer.fit(df['Text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25,
                                                   random_state= 42)

In [95]:
X_train = vectorizer.transform(X_train.values)

In [96]:
X_test = vectorizer.transform(X_test.values)

In [97]:
X_train.shape

(3750, 1000)

In [102]:
model = Sequential()
model.add(layers.Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(layers.Dense(50, activation='softmax'))

In [104]:
model.compile(loss='categorical_crossentropy', 
               optimizer= 'adam', 
            metrics=['accuracy'])

#Using Adam optimizer and the categorical cross-entropy loss function
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
Total params: 105,150
Trainable params: 105,150
Non-trainable params: 0
_________________________________________________________________


In [100]:
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [105]:
history = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=True,
                     validation_data=(X_test, y_test),
                    batch_size=10)

Train on 3750 samples, validate on 1250 samples
Epoch 1/100
3750/3750 [==============================] - 4s 1ms/step - loss: 3.2855 - acc: 0.3576 - val_loss: 2.3244 - val_acc: 0.5968
Epoch 2/100
3750/3750 [==============================] - 3s 814us/step - loss: 1.6803 - acc: 0.6581 - val_loss: 1.3697 - val_acc: 0.6488
Epoch 3/100
3750/3750 [==============================] - 3s 820us/step - loss: 1.0517 - acc: 0.7677 - val_loss: 1.0623 - val_acc: 0.7048
Epoch 4/100
3750/3750 [==============================] - 3s 822us/step - loss: 0.7697 - acc: 0.8224 - val_loss: 0.8958 - val_acc: 0.7440
Epoch 5/100
3750/3750 [==============================] - 3s 821us/step - loss: 0.5938 - acc: 0.8664 - val_loss: 0.7951 - val_acc: 0.7744
Epoch 6/100
3750/3750 [==============================] - 3s 807us/step - loss: 0.4639 - acc: 0.8960 - val_loss: 0.7298 - val_acc: 0.7800
Epoch 7/100
3750/3750 [==============================] - 3s 791us/step - loss: 0.3679 - acc: 0.9219 - val_loss: 0.6751 - val_acc: 0.

3750/3750 [==============================] - 2s 584us/step - loss: 2.6065e-05 - acc: 1.0000 - val_loss: 0.8595 - val_acc: 0.8328
Epoch 60/100
3750/3750 [==============================] - 2s 582us/step - loss: 2.2780e-05 - acc: 1.0000 - val_loss: 0.8704 - val_acc: 0.8352
Epoch 61/100
3750/3750 [==============================] - 2s 579us/step - loss: 2.0552e-05 - acc: 1.0000 - val_loss: 0.8800 - val_acc: 0.8328
Epoch 62/100
3750/3750 [==============================] - 2s 591us/step - loss: 1.7390e-05 - acc: 1.0000 - val_loss: 0.8854 - val_acc: 0.8360
Epoch 63/100
3750/3750 [==============================] - 2s 581us/step - loss: 1.6021e-05 - acc: 1.0000 - val_loss: 0.9039 - val_acc: 0.8328
Epoch 64/100
3750/3750 [==============================] - 2s 582us/step - loss: 1.4083e-05 - acc: 1.0000 - val_loss: 0.9041 - val_acc: 0.8376
Epoch 65/100
3750/3750 [==============================] - 2s 580us/step - loss: 1.2330e-05 - acc: 1.0000 - val_loss: 0.9147 - val_acc: 0.8360
Epoch 66/100
3750/3

In [106]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.8288


In [ ]:
NEURAL NETWORK WITH A SINGLE HIDDEN LAYER AND SOFTMAX AT THE OUTPUT LAYER WITH THE TF-IDF VECTORIZER ACHIEVES 73.84% ACCURACY

In [108]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [109]:

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
#Trying a model with two hidden layers:

In [25]:
model_two = Sequential()
model_two.add(layers.Dense(100,input_dim = input_dimension, activation = 'relu'))
model_two.add(layers.Dense(75,activation = 'relu'))
model_two.add(layers.Dense(50, activation='softmax'))

In [27]:
model_two.compile(loss='categorical_crossentropy', 
               optimizer='adam', 
            metrics=['accuracy'])


In [28]:
history = model_two.fit(X_train, y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(X_test, y_test),
                    batch_size=10)

In [29]:
model_two.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               44500     
_________________________________________________________________
dense_3 (Dense)              (None, 75)                7575      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                3800      
Total params: 55,875
Trainable params: 55,875
Non-trainable params: 0
_________________________________________________________________


In [30]:
model_two.evaluate(X_train,y_train)

3750/3750 [==============================] - 1s 176us/step


[0.0007415856130576382, 0.9997333333333334]

In [31]:
model_two.evaluate(X_test,y_test)

1250/1250 [==============================] - 0s 100us/step


[2.1614733142852782, 0.7016]

In [ ]:
#As the validation accuracy is quite a bit greater than the training accuracy, this suggests that we the model has overfitted.

In [32]:
#We will now try a simpler model, with the aim of brining down the validation loss to somewhere closer to the training data loss

In [34]:
model_three = Sequential()
model_three.add(layers.Dense(100,input_dim = input_dimension, activation = 'relu'))
model_three.add(layers.Dense(75,activation = 'relu'))
model_three.add(layers.Dense(50, activation='softmax'))

In [36]:
model_three.compile(loss='categorical_crossentropy', 
               optimizer='sgd', 
            metrics=['accuracy'])

In [37]:
#The model also tests the accuracy on the validation data set per epoch

history = model_three.fit(X_train, y_train,
                     epochs=200,
                     verbose=False,
                     validation_data=(X_test, y_test),
                    batch_size=10)

In [38]:
model_three.evaluate(X_train,y_train)

3750/3750 [==============================] - 1s 186us/step


[0.0015797486290956538, 1.0]

In [39]:
model_three.evaluate(X_test,y_test)

1250/1250 [==============================] - 0s 368us/step


[1.7865182443752885, 0.704]

In [49]:
X_train.toarray().shape

(3750, 444)